This notebook was automatically generated from your MAST-ML run so you can recreate the
plots. Some things are a bit different from the usual way of creating plots - we are
using the [object oriented
interface](https://matplotlib.org/tutorials/introductory/lifecycle.html) instead of
pyplot to create the `fig` and `ax` instances.


In [ ]:
"""
This module contains a collection of functions which make plots (saved as png files) using matplotlib, generated from
some model fits and cross-validation evaluation within a MAST-ML run.

This module also contains a method to create python notebooks containing plotted data and the relevant source code from
this module, to enable the user to make their own modifications to the created plots in a straightforward way (useful for
tweaking plots for a presentation or publication).
"""

import math
import os
import pandas as pd
import itertools
import warnings
import logging
from collections import Iterable
from os.path import join
from collections import OrderedDict

# Ignore the harmless warning about the gelsd driver on mac.
warnings.filterwarnings(action="ignore", module="scipy",
                        message="^internal gelsd")
# Ignore matplotlib deprecation warning (set as all warnings for now)
warnings.filterwarnings(action="ignore")

import numpy as np
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve

import matplotlib
from matplotlib import pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure, figaspect
from matplotlib.animation import FuncAnimation
from matplotlib.font_manager import FontProperties
import matplotlib.mlab as mlab
from scipy.stats import gaussian_kde
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes

# Needed imports for ipynb_maker
from mastml.utils import nice_range
from mastml.metrics import nice_names

import inspect
import textwrap
from pandas import DataFrame, Series

import nbformat

from functools import wraps

matplotlib.rc('font', size=18, family='sans-serif') # set all font to bigger
matplotlib.rc('figure', autolayout=True) # turn on autolayout

# adding dpi as a constant global so it can be changed later
DPI = 250

log = logging.getLogger() # only used inside ipynb_maker I guess



In [ ]:
def stat_to_string(name, value):
    """
    Method that converts a metric object into a string for displaying on a plot

    Args:

        name: (str), long name of a stat metric or quantity

        value: (float), value of the metric or quantity

    Return:

        (str), a string of the metric name, adjusted to look nicer for inclusion on a plot

    """

    " Stringifies the name value pair for display within a plot "
    if name in nice_names:
        name = nice_names[name]
    else:
        name = name.replace('_', ' ')

    # has a name only
    if not value:
        return name
    # has a mean and std
    if isinstance(value, tuple):
        mean, std = value
        return f'{name}:' + '\n\t' + f'{mean:.3f}' + r'$\pm$' + f'{std:.3f}'
    # has a name and value only
    if isinstance(value, int) or (isinstance(value, float) and value%1 == 0):
        return f'{name}: {int(value)}'
    if isinstance(value, float):
        return f'{name}: {value:.3f}'
    return f'{name}: {value}' # probably a string


def plot_stats(fig, stats, x_align=0.65, y_align=0.90, font_dict=dict(), fontsize=14):
    """
    Method that prints stats onto the plot. Goes off screen if they are too long or too many in number.

    Args:

        fig: (matplotlib figure object), a matplotlib figure object

        stats: (dict), dict of statistics to be included with a plot

        x_align: (float), float denoting x position of where to align display of stats on a plot

        y_align: (float), float denoting y position of where to align display of stats on a plot

        font_dict: (dict), dict of matplotlib font options to alter display of stats on plot

        fontsize: (int), the fontsize of stats to display on plot

    Returns:

        None

    """

    stat_str = '\n'.join(stat_to_string(name, value)
                           for name,value in stats.items())

    fig.text(x_align, y_align, stat_str,
             verticalalignment='top', wrap=True, fontdict=font_dict, fontproperties=FontProperties(size=fontsize))


def make_fig_ax(aspect_ratio=0.5, x_align=0.65, left=0.10):
    """
    Method to make matplotlib figure and axes objects. Using Object Oriented interface from https://matplotlib.org/gallery/api/agg_oo_sgskip.html

    Args:

        aspect_ratio: (float), aspect ratio for figure and axes creation

        x_align: (float), x position to draw edge of figure. Needed so can display stats alongside plot

        left: (float), the leftmost position to draw edge of figure

    Returns:

        fig: (matplotlib fig object), a matplotlib figure object with the specified aspect ratio

        ax: (matplotlib ax object), a matplotlib axes object with the specified aspect ratio

    """
    # Set image aspect ratio:
    w, h = figaspect(aspect_ratio)
    fig = Figure(figsize=(w,h))
    FigureCanvas(fig)

    # Set custom positioning, see this guide for more details:
    # https://python4astronomers.github.io/plotting/advanced.html
    #left   = 0.10
    bottom = 0.15
    right  = 0.01
    top    = 0.05
    width = x_align - left - right
    height = 1 - bottom - top
    ax = fig.add_axes((left, bottom, width, height), frameon=True)
    fig.set_tight_layout(False)
    
    return fig, ax


In [ ]:
def plot_residuals_histogram(y_true, y_pred, savepath,
                             stats, title='residuals histogram', label='residuals'):
    """
    Method to calculate and plot the histogram of residuals from regression model

    Args:

        y_true: (numpy array), array of true y data values

        y_pred: (numpy array), array of predicted y data values

        savepath: (str), path to save the plotted precision-recall curve

        stats: (dict), dict of training or testing statistics for a particular run

        title: (str), title of residuals histogram

        label: (str), label used for axis labeling

    Returns:

        None

    """

    # make fig and ax, use x_align when placing text so things don't overlap
    x_align = 0.64
    fig, ax = make_fig_ax(x_align=x_align)

    #ax.set_title(title)
    # do the actual plotting
    residuals = y_true - y_pred

    #Output residuals data and stats to spreadsheet
    path = os.path.dirname(savepath)
    pd.DataFrame(residuals).describe().to_csv(os.path.join(path,'residual_statistics.csv'))
    pd.DataFrame(residuals).to_csv(path+'/'+'residuals.csv')

    #Get num_bins using smarter method
    num_bins = get_histogram_bins(y_df=residuals)
    ax.hist(residuals, bins=num_bins, color='b', edgecolor='k')

    # normal text stuff
    ax.set_xlabel('Value of '+label, fontsize=16)
    ax.set_ylabel('Number of occurences', fontsize=16)

    # make y axis ints, because it is discrete
    #ax.yaxis.set_major_locator(MaxNLocator(integer=True))

    plot_stats(fig, stats, x_align=x_align, y_align=0.90)
    plot_stats(fig, pd.DataFrame(residuals).describe().to_dict()[0], x_align=x_align, y_align=0.60)

    fig.savefig(savepath, dpi=DPI, bbox_inches='tight')


In [ ]:
from numpy import array
from collections import OrderedDict
from io import StringIO
y_true = array([177.28302038, 150.37102595, 325.49788774, 249.54944341,
       210.37520634, 200.26888144, 276.21736875, 193.03789201,
       346.91290112,  79.88432553,  64.76678569, 146.59493555,
       140.87646025,  96.27921014, 186.83577486, 176.19335258,
       111.98936866, 149.58971125, 147.4918845 , 181.75592179,
       200.50453822, 125.08123493, 335.55187932, 124.02294655,
       116.8908945 , 127.740217  ,  72.36735708,  45.7531836 ,
        58.99391702,   0.        ,  63.04573585,  24.184234  ,
        24.89260867,  25.66391325, 390.4260105 ,  70.97342159,
       129.12841409,  89.72129861, 100.97112048, 636.33932472,
        68.67920135,  67.13987502,  78.67122954, 157.7155851 ,
        92.54373354, 227.4912848 , 113.13530223, 102.03643628,
       128.36434487, 238.33422924,   0.        , 127.68188418,
       347.3831    ,  29.45910878, 156.62793026, 114.52532705,
         0.        ,  63.15129713,  64.50496061, 357.16072876,
        26.88964985,  53.828979  ,   0.        ,  10.5881855 ,
        66.14597382,   0.        ,  50.49019186,  41.17090553,
        43.19281587,  48.91154301,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,  32.07826851,
        26.82825386,  22.3715509 ,  75.38976676,  26.31847508,
         0.        ,   0.        ,  60.36059818, 108.22466848,
        25.17480383,  40.61873268,  43.36372073,  56.75173357,
        40.25443982,  39.24127519,  36.50751242,  49.84773514,
        40.26469461,   0.        ,  31.3433527 ,  19.2047597 ,
       210.80892907, 108.97985761, 115.20029363,   0.        ,
         0.        , 100.45704471,  30.22875   ,   0.        ,
         0.        ,   0.        ,  16.20435918,  44.41847336,
         0.        ,  50.8263675 ,  88.38029969, 118.54297288,
        30.17024475,   0.        ,  47.26926175,  23.66260283,
        31.378064  ,  35.15839525,   0.        , 105.69316532,
       144.19457152,  78.00949578, 107.97506678,  86.53565683,
       112.57384503,  77.98044279,  26.83259762,   0.        ,
       100.94922471, 187.39104625, 468.0346095 ,  59.1838239 ,
        73.18129625, 125.29220761,  89.81673775,  69.97238548,
         0.        ,  15.00782838, 114.55464449, 258.66530956,
       276.02872083, 109.03341082, 106.70233503,  99.7625282 ,
       294.4157031 , 165.77843319, 197.24625694, 202.28394837,
       230.92190461, 113.90148661,   0.        ,  47.45866309,
        38.14651548,  64.97491856, 100.44019458,   9.106651  ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,  51.31971375,
        50.94866855,  47.54553553, 155.46507812, 132.77386341,
        75.46581208,  74.56373534, 135.99188516,  66.64638295,
        43.1541225 ,  37.473804  ,  58.73687405,  37.03306019,
        56.73302619,  34.01111394,  40.76042242,  65.03154095,
       162.89217552, 169.71976527,   0.        , 152.42942927,
       195.44299332, 102.15127111,  81.59092701,  83.02690466,
       106.70124669,   0.        , 141.37372586,  40.06738694,
        26.74964638,  71.67773168,  21.819592  ,  55.01006219,
        44.01037784, 137.88193919,  95.49667394, 105.44236844,
        19.80860463,  21.12324531,   7.05592531,   0.        ,
         0.        ,   0.        ,   0.        , 157.24298149,
       265.27059207, 209.0198447 , 193.22945633, 179.65413218,
       168.86730174, 275.65617815, 366.50308111,  91.53400436,
       116.18097654, 178.93978486, 136.29376713, 244.14891128,
       122.77257944, 220.00338082, 139.01358071, 192.28465626,
       144.70871262, 191.8366738 , 141.5352297 , 144.0065577 ,
       190.07525544, 106.22035875, 101.76810798, 103.16515278,
       121.42897728, 156.13605672, 231.128112  , 135.39084503,
       250.57926852, 367.1832279 , 357.14535132, 237.06234447,
       273.61317932, 362.46105447, 294.31041312, 323.82263694,
       221.05590358, 189.64836459, 204.04657134, 223.81669945,
       213.49368171, 255.47898307, 337.2143761 , 187.89440857,
        76.49603762, 111.02632779, 156.18544229,  70.43789669,
       149.64777796, 212.75663082, 135.16475661, 151.35614397,
       252.16135397, 103.45613548, 154.6418111 ,  98.39446077,
       192.71275543,  95.45473206, 183.44919008, 102.79458377,
       121.12739898, 353.34957447, 338.95548037, 258.6400468 ,
       317.205543  , 350.05257055, 317.35687145, 252.61149593,
       218.13728473, 141.45345548,  89.57790862, 142.82619229,
        84.93486971, 104.45964543, 134.59960205, 179.65931277,
       213.36565687, 244.80034507, 165.1097297 ,  54.1587672 ,
       150.80186342, 158.4134052 ,  86.8301581 , 238.53505817,
       202.52553418, 210.32978743, 227.71776104, 313.927543  ,
       183.57299247, 318.78535376, 845.33730031, 857.23166531,
       110.4724554 , 101.8838757 ,  67.71649743,  81.00509637,
       138.84499588,  77.08989732, 104.682468  , 143.72236588,
       190.04640497,  94.39134167, 130.56858186, 125.30956249,
       102.21705765, 110.48019418, 113.32195047,  43.01592814,
        75.85284451,  27.21757246,  32.77341137, 135.8400834 ,
       121.55357738, 167.32480457, 172.75454393, 163.40928559,
       165.12741257, 184.78773175, 163.98220844, 137.40668645,
        29.88706053, 123.93596706, 159.40354586, 100.81566362,
       258.82805093, 104.37297146, 137.58344095, 117.14316894,
       162.41908077, 144.11116592,  89.81673775,  87.6081324 ,
        66.96310632,  66.99094092,  36.36673937,  31.31275089,
        72.94226156, 198.66504686,  35.28329862,  35.04593958,
       105.85969748, 143.07547478,  52.87341873,  99.86697643,
       192.00928341,  10.43143757,   0.        ,  87.08840286,
       178.94455909,  35.07038698, 110.11022338,   6.92239663,
        93.84230286,  21.50586649,  17.84640765,   0.        ,
         0.        ,  62.44816555, 121.97489618,  28.36967345,
         0.        ,  51.91571301,   0.        ,  93.06800775,
        21.53240718,  28.66375885,   7.99376675,   0.        ,
        28.40700128,   0.        ,   0.        ,  14.17874738,
        33.75589221, 184.38740476, 267.85442553, 139.06215854,
       207.75718308,  75.14562075,  76.84371871,  79.07980342,
        82.90154417,  67.41487645,  36.096681  ,  70.45416536,
        38.19251136,   0.        ,  42.38549807,  52.55044553,
        38.17424353,  38.38155275,  40.23930643,   0.        ,
         0.        , 103.73141473, 123.84952014, 166.34766857,
        82.62024503, 157.61174657,  17.70199531,  67.66092594,
        84.87592237,  31.26186719,  25.95621478,  59.06262229,
       160.39510805,  54.02604304,   0.        ,  78.06700275,
        99.02368612,  65.77979603,  21.912008  ,  71.75494749,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ])
y_pred = array([ 2.26427993e+02,  1.61508850e+02,  2.78709183e+02,  2.96055061e+02,
        2.05722422e+02,  1.84643693e+02,  1.73217919e+02,  1.58852699e+02,
        2.79774165e+02,  1.06034432e+02,  1.06047797e+02,  1.57171644e+02,
        1.54746524e+02,  9.50526721e+01,  1.47314390e+02,  1.67526501e+02,
        1.82217242e+02,  1.57148539e+02,  1.20012977e+02,  1.17232069e+02,
        2.17060053e+02,  1.51663588e+02,  2.63111629e+02,  1.51775243e+02,
        1.07568203e+02,  1.24542873e+02,  9.12823018e+01,  7.52176827e+01,
        1.03705493e+02,  3.56639533e+01,  2.86621919e+01,  6.45872531e+01,
        5.48901109e+01,  5.23964208e+01,  2.79032494e+02,  1.05045009e+02,
        9.33856558e+01,  9.59920210e+01,  9.83535927e+01,  5.15586990e+02,
        8.15187405e+01,  6.62350120e+01,  7.39380558e+01,  9.47718727e+01,
        1.10554497e+02,  1.32225070e+02,  1.31801813e+02,  1.50475038e+02,
        1.64575148e+02,  1.76159481e+02,  1.15599097e+02,  1.20434913e+02,
        1.79300236e+02,  1.12298132e+02,  8.28533102e+01,  2.03753069e+02,
        9.17556308e+01,  1.16110260e+02,  1.18134314e+02,  2.86194991e+02,
        7.03327236e+01,  3.78537363e+01,  3.52831928e+01,  4.04722114e+01,
        7.83099221e+01,  2.14379475e+01,  5.93585154e+01,  4.93337234e+01,
        4.46458633e+01,  6.82915704e+01, -2.00482333e+00,  2.99234580e+01,
        7.47358543e+01,  4.86157232e+01,  1.98857290e+01,  3.29919922e+01,
        2.31906056e+01,  2.80239565e+01,  2.17749996e+01,  8.37728880e+01,
        5.05045871e+01,  5.20523080e+01,  8.39898960e+01,  9.38100971e+01,
        4.75787925e+01,  6.85971227e+01,  5.85905664e+01,  6.91345467e+01,
        5.76588136e+01,  8.79347003e+01,  5.12877802e+01,  3.82124235e+01,
        3.21477926e+01,  6.29717054e+01,  4.72233890e+01,  2.98701973e+01,
        3.23672619e+01,  1.79403289e+01,  7.31089554e+01,  7.01987907e+01,
        2.47863372e+02,  1.01936226e+02,  9.54269727e+01,  4.59300505e+01,
        7.79583011e+01,  9.08747148e+01,  2.10579959e+01,  1.68418131e+01,
        4.63766131e+01,  2.35227428e+01,  7.89417308e+01,  7.46598042e+01,
        2.59440844e+01,  3.86971109e+01,  6.14290506e+01,  4.81452998e+01,
        2.37102683e+01,  2.55043428e+01,  2.99546285e+01,  2.24459888e+01,
        4.55171876e+01,  4.07095142e+01,  1.45934236e+00,  1.18555786e+02,
        1.42114320e+02,  6.86046741e+01,  9.34122216e+01,  8.13247142e+01,
        1.18905799e+02,  1.07533629e+02,  3.09957497e+01,  4.60614918e+01,
        2.05604790e+02,  1.26529550e+02,  2.78810771e+02,  4.84560000e+01,
        1.23454072e-04,  2.42641012e+02,  7.67242276e+01,  7.49741722e+01,
        7.06676972e+01,  1.29827251e+02,  1.68050217e+02,  2.81667100e+02,
        2.18355522e+02,  1.86499787e+02,  1.26061066e+02,  2.30466851e+01,
        2.36890496e+02,  6.91262804e+01,  1.03797189e+02,  4.21132689e+01,
        7.84061307e+01,  1.10292872e+02,  5.50717625e+01,  8.90120930e+01,
        7.62140327e+01,  9.82210623e+01,  6.28391929e+01,  2.09249797e+01,
        1.74729866e+01,  1.11218136e+01,  6.95700026e+00,  1.24998775e+01,
        2.41094862e+01,  3.59837630e+01,  2.86352387e+01,  7.12846623e+01,
        3.61068502e+01,  1.90966699e+01,  1.16072002e+02,  1.08044207e+02,
        8.77105287e+01,  7.01111556e+01,  1.08969119e+02,  1.36536956e+02,
        7.54111596e+01,  9.29520383e+01,  7.45564232e+01,  6.47899826e+01,
        6.07860533e+01,  6.56796914e+01,  2.04628590e+01,  2.49410783e+01,
        1.77356794e+02,  1.66463249e+02,  3.24906188e+01,  1.68066266e+02,
        2.35176470e+02,  1.14577699e+02,  1.04821720e+02,  7.10472716e+01,
        1.07512388e+02,  4.53511651e+01,  8.66638635e+01,  3.49765385e+01,
        3.09531617e+01,  8.11522199e+01,  2.92338710e+01,  5.09473743e+01,
        3.73818873e+01,  4.15427029e+01,  1.84858300e+01,  2.24336331e+01,
       -9.45238030e-01,  2.12347575e+01,  2.56179249e+01,  4.15117886e+01,
        1.26701593e+01,  1.62589840e+01,  2.85182204e+01,  1.39823278e+02,
        3.06414409e+02,  2.67765569e+02,  1.60443745e+02,  1.47329145e+02,
        1.28855044e+02,  1.81107833e+02,  2.93929805e+02,  1.44745709e+02,
        1.94303432e+02,  2.51396444e+02,  8.98544004e+01,  1.04683068e+02,
        9.10737511e+01,  2.21474607e+02,  1.65902872e+02,  1.87814373e+02,
        1.35138183e+02,  1.40010843e+02,  1.32396571e+02,  9.84745204e+01,
        1.39008605e+02,  6.44264198e+01,  6.41451204e+01,  1.37570416e+02,
        1.21016372e+02,  1.60239449e+02,  1.59673355e+02,  1.22154551e+02,
        2.51274467e+02,  2.68055211e+02,  3.04480816e+02,  2.20304053e+02,
        2.98522217e+02,  2.51121984e+02,  2.77445191e+02,  3.04989379e+02,
        2.40544427e+02,  1.91883923e+02,  2.16804538e+02,  2.75453684e+02,
        2.13948326e+02,  2.96403670e+02,  2.06775318e+02,  1.93363631e+02,
        8.46844756e+01,  8.64486452e+01,  1.55102988e+02,  8.53096584e+01,
        1.41820072e+02,  2.19136757e+02,  1.51896185e+02,  1.53337289e+02,
        1.69998521e+02,  8.13000335e+01,  1.31223879e+02,  7.97892101e+01,
        1.39920501e+02,  6.27666113e+01,  7.34401748e+01,  1.32257657e+02,
        1.15149808e+02,  2.45941868e+02,  2.31838533e+02,  2.25908808e+02,
        2.96075018e+02,  2.32589238e+02,  2.88046749e+02,  2.21682989e+02,
        1.68196419e+02,  1.61266460e+02,  8.71049239e+01,  1.49215894e+02,
        8.85083162e+01,  8.66214153e+01,  1.52479631e+02,  2.04540910e+02,
        2.26094976e+02,  2.58672964e+02,  1.37282947e+02,  7.64317098e+01,
        7.46394287e+01,  1.59429673e+02,  1.12337765e+02,  1.75766785e+02,
        2.30544615e+02,  2.14716004e+02,  2.68306291e+02,  2.95428251e+02,
        1.95679899e+02,  2.26087505e+02,  3.92195627e+02,  3.41814080e+02,
        1.39662475e+02,  1.32410949e+02,  1.21990988e+02,  1.16143118e+02,
        1.16495893e+02,  1.18421393e+02,  1.19243905e+02,  1.41331346e+02,
        1.51647145e+02,  1.28061014e+02,  1.36660038e+02,  1.32559798e+02,
        1.28296606e+02,  1.21125191e+02,  1.29242812e+02,  6.41580577e+01,
        1.17331086e+02,  3.47904198e+01,  3.88986957e+01,  2.39531402e+02,
        1.68876917e+02,  1.13242537e+02,  1.83269431e+02,  1.76644763e+02,
        1.79072976e+02,  1.81636209e+02,  1.31522535e+02,  1.14838228e+02,
        5.57047634e+01,  1.47898933e+02,  1.58861692e+02,  1.20695914e+02,
        3.08092499e+02,  1.10528471e+02,  1.61562937e+02,  1.25997817e+02,
        1.56965945e+02,  1.30208133e+02,  7.67242276e+01,  1.10888641e+02,
        7.53832542e+01,  7.01170847e+01,  3.70938162e+01,  5.36145661e+01,
        9.23819440e+01,  1.54259062e+02,  4.02930285e+01,  1.73034916e+01,
        8.58036818e+01,  5.45681295e+01,  7.23336579e+01,  9.66429914e+01,
        1.39608700e+02,  5.53575321e+01,  4.41548103e+01,  6.74417675e+01,
        9.99457746e+01,  1.28155736e+02,  1.53445559e+02,  3.02032695e+01,
        5.77803538e+01,  3.88947951e+01,  6.35777737e+01, -1.63616140e+00,
       -5.96603368e-01,  4.87736238e+01,  5.56241623e+01,  3.74699195e+01,
        6.02661320e+01,  5.98874976e+01,  2.06659466e+00,  6.40512936e+01,
        5.23755014e+01,  4.91176446e+01,  2.47458250e+01,  3.16970061e+01,
        7.57185163e+01, -3.61534992e+00, -1.35912073e+00,  2.31263756e+01,
        2.71037103e+01,  1.15732664e+02,  1.61095634e+02,  9.84834488e+01,
        1.47335472e+02,  7.78687184e+01,  4.30568375e+01,  5.10251750e+01,
        6.03311607e+01,  9.75898316e+01,  7.49139803e+01,  4.65187176e+01,
        4.65630741e+01, -6.64056062e+00,  2.79039561e+01,  6.44314082e+01,
        4.49229179e+01,  2.93514790e+01,  2.81281458e+01,  1.86658223e+01,
        3.26344256e+01,  7.76634622e+01,  1.42429021e+02,  1.97803542e+02,
        8.80900283e+01,  2.09449105e+02,  5.66222599e+01,  7.02426972e+01,
        4.69240394e+01,  2.31886414e+01,  3.63664702e+01,  3.28426605e+01,
        9.52808090e+01,  3.76672895e+01,  2.67782550e+01,  3.14474561e+01,
        4.81607032e+01,  3.20124821e+01,  6.90179522e+01,  3.01825001e+01,
        1.98662606e+01,  2.55693497e+01,  2.14396212e+01,  1.41898495e+01,
       -1.77571398e+00])
savepath = '/home/nerve/Desktop/skunkworks/lane_schultz/learning_curves/learning/StandardScaler/SequentialFeatureSelector/KernelRidge/RepeatedKFold/split_1/test_residuals_histogram.png'
stats = OrderedDict([('R2', 0.7388745686948082), ('root_mean_squared_error', 55.886681648930875), ('mean_absolute_error', 35.28271968611685), ('rmse_over_stdev', 0.6048305666835216)])
title = 'test_residuals_histogram'
label = 'Energy above convex hull (meV/atom)'

In [ ]:
import pandas as pd
from IPython.display import Image, display

plot_residuals_histogram(y_true, y_pred, savepath, stats, title, label)
display(Image(filename='test_residuals_histogram.png'))
